In [0]:
import gc
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
random.seed(42)

In [0]:
def reload():
    gc.collect()
    df = pd.read_csv('C:\\Users\\ymo\\Desktop\\502FP\\train_V2.csv')
    invalid_match_ids = df[df['winPlacePerc'].isna()]['matchId'].values
    df = df[-df['matchId'].isin(invalid_match_ids)]
    return df

In [0]:
def rank_by_team(df):
    cols_to_drop = ['Id', 'groupId', 'matchId', 'winPlacePerc','matchType']
    features = [col for col in df.columns if col not in cols_to_drop]
    agg = df.groupby(['matchId', 'groupId'])[features].mean()
    agg = agg.groupby('matchId')[features].rank(pct=True)
    return df.merge(agg, suffixes=['', '_mean_rank'], how='left', on=['matchId', 'groupId'])
 
def median_by_team(df):
    cols_to_drop = ['Id', 'groupId', 'matchId', 'winPlacePerc', 'matchType']
    features = [col for col in df.columns if col not in cols_to_drop and '_mean_rank' not in col]
    agg = df.groupby(['matchId', 'groupId'])[features].median()
    return df.merge(agg, suffixes=['', '_median'], how='left', on=['matchId', 'groupId'])

def rank_and_median(df):
    df=rank_by_team(df)
    return median_by_team(df)

def run_experiment(preprocess):
    
    df = reload()
    
    
    df = preprocess(df)

    target = 'winPlacePerc'
    cols_to_drop = ['Id', 'groupId', 'matchId', target, 'matchType']
    cols_to_fit = [col for col in df.columns if col not in cols_to_drop]
    #cols_to_fit=[col for col in df.columns if col[-4:]=='rank']

    

    model = RandomForestRegressor(n_estimators=40,min_samples_leaf=10,max_features='sqrt',n_jobs=-1)
    scores = cross_val_score(model, df[cols_to_fit], df[target], cv=5, scoring='neg_mean_absolute_error')
    
    return str(scores.mean())[1:9] + '+/-' + str(scores.std()*2)[0:9]

In [0]:
run_experiment(rank_and_median)